In [1]:
import pandas as pd
import numpy as np
import os,sys
import json
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/train.csv')
test_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/test.csv') 

In [3]:
df = train_df.append(test_df, ignore_index=True)

In [ ]:
df.iloc[0]

In [4]:
data_text_t1_en = df[['title1_en']] #+ test_df[['title1_en']]
data_text_t1_en['index'] = df.index
documents_t1 = data_text_t1_en

data_text_t2_en = df[['title2_en']] #+ test_df[['title2_en']]
data_text_t2_en['index'] = df.index
documents_t2 = data_text_t2_en

/Users/Administrator/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Administrator/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
#data_text_t1_en = train_df[['title1_en']] #+ test_df[['title1_en']]
#data_text_t1_en['index'] = train_df.index
#documents_t1 = data_text_t1_en

In [ ]:
#data_text_t2_en = train_df[['title2_en']] #+ test_df[['title2_en']]
#data_text_t2_en['index'] = train_df.index
#documents_t2 = data_text_t2_en

In [ ]:
data_text_t1_en_test = test_df[['title1_en']]
data_text_t1_en_test['index'] = test_df.index
documents_t1_test = data_text_t1_en_test
data_text_t2_en_test = test_df[['title2_en']]
data_text_t2_en_test['index'] = test_df.index
documents_t2_test = data_text_t2_en_test

In [5]:
#docs = documents_t1['title1_en'] + documents_t2['title2_en']
documents_t2.columns = ['title1_en', 'index']

In [6]:
docs = documents_t1.append(documents_t2, ignore_index=True)

In [ ]:
#docs_test = documents_t1_test['title1_en'] + documents_t2_test['title2_en']

In [ ]:
#docs_merged = docs.append(docs_test)

In [9]:
documents_t1.shape

(400678, 2)

In [ ]:
#docs_test.shape

In [ ]:
#docs_merged.shape

In [10]:
type(docs)

pandas.core.series.Series

In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Administrator/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
processed_docs_merged = docs['title1_en'].map(preprocess)
processed_docs_merged[:10]

0                 [insur, benefit, peopl, rural, area]
1    [come, shenzhen, sooner, later, come, year, sh...
2    [come, shenzhen, sooner, later, come, year, sh...
3    [come, shenzhen, sooner, later, come, year, sh...
4                    [discrimin, gutter, mean, garlic]
5    [come, shenzhen, sooner, later, come, year, sh...
6                                [durian, kill, wrong]
7    [come, shenzhen, sooner, later, come, year, sh...
8              [frog, frog, fertil, test, play, jewel]
9                    [discrimin, gutter, mean, garlic]
Name: title1_en, dtype: object

In [ ]:
processed_docs_merged

In [ ]:
processed_docs_merged[0]

In [23]:
for i in range(0, len(processed_docs_merged)):
    processed_docs_merged[i] = ' '.join(processed_docs_merged[i])


In [19]:
len(processed_docs_merged)/2

400678.0

In [17]:
 y = train_df.loc[:,'label']

In [18]:
y_new =[0] * len(y)
for i in range(len(y)):
    if y[i] == 'unrelated':
        y_new[i] = 0
    elif y[i] == 'agreed':
        y_new[i] = 1
    elif y[i] == 'disagreed':
        y_new[i] = 2

In [29]:
type(processed_docs_merged[0])

str

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline


vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)

svd_model = TruncatedSVD(n_components=100,         
                         algorithm='randomized',
                         n_iter=10)

svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])

svd_matrix = svd_transformer.fit_transform(processed_docs_merged[:400678])

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline


vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True, ngram_range =(1,2))

svd_model = TruncatedSVD(n_components=100,         
                         algorithm='randomized',
                         n_iter=10)

svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])

svd_matrix = svd_transformer.fit_transform(processed_docs_merged)

In [32]:
svd_matrix

array([[ 0.01918863,  0.0286159 ,  0.01808505, ...,  0.011052  ,
         0.00617448,  0.02045532],
       [ 0.02029697,  0.02127209,  0.01619079, ...,  0.00135781,
        -0.02195548, -0.0591473 ],
       [ 0.02029697,  0.02127209,  0.01619079, ...,  0.00135781,
        -0.02195548, -0.0591473 ],
       ...,
       [ 0.00780833,  0.00961229,  0.00345061, ...,  0.01590715,
        -0.00776811,  0.01904509],
       [ 0.01671944,  0.01668049,  0.01201423, ..., -0.00030251,
        -0.0029016 ,  0.00866314],
       [ 0.00617216,  0.00717253,  0.00483158, ..., -0.01653337,
         0.0034951 ,  0.02379681]])

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVectorizer?

In [28]:
svd_matrix.shape

(320552, 100)

In [29]:
X = svd_matrix

In [30]:
X.shape

(320552, 100)

In [ ]:
data_text_t1_en_test = test_df[['title1_en']]
data_text_t1_en_test['index'] = test_df.index
documents_t1_test = data_text_t1_en_test
data_text_t2_en_test = test_df[['title2_en']]
data_text_t2_en_test['index'] = test_df.index
documents_t2_test = data_text_t2_en_test

In [ ]:
docs_test = documents_t1_test['title1_en'] + documents_t2_test['title2_en']

In [ ]:
processed_docs_test = docs_test.map(preprocess)
processed_docs_test[:10]

In [ ]:
for i in range(0, len(processed_docs_test)):
    processed_docs_test[i] = ' '.join(processed_docs_test[i])

In [ ]:
svd_matrix_test = svd_transformer.fit_transform(processed_docs_test)

In [ ]:
len(y_new)

In [45]:
X

array([[0.09465792, 0.07959106, 0.0855641 , ..., 0.03435419, 0.00522916,
        0.01169125],
       [0.05635475, 0.04080039, 0.04651873, ..., 0.02997685, 0.04049267,
        0.00927411],
       [0.056504  , 0.03409208, 0.03553103, ..., 0.04189932, 0.04062652,
        0.00884206],
       ...,
       [0.01180759, 0.00943695, 0.01055401, ..., 0.00482765, 0.00932629,
        0.03539339],
       [0.02676953, 0.02180736, 0.02464082, ..., 0.00247275, 0.00778814,
        0.01658776],
       [0.02465559, 0.02223488, 0.02647584, ..., 0.02951702, 0.00552991,
        0.06344828]])

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size = 0.20, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(abs(X_train), y_train)

# Fitting Logistic Regression to the Training set
#from sklearn.linear_model import LogisticRegression
#classifier = LogisticRegression(random_state = None, multi_class = 'multinomial', class_weight = 'balanced', solver = 'newton-cg')
#classifier.fit(X_train, y_train)

#from sklearn.ensemble import RandomForestClassifier
#classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
#classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)




In [39]:
from sklearn.naive_bayes import MultinomialNB
MultinomialNB?

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier?

In [32]:
cm

array([[41410,  2520,     0],
       [14906,  3608,     0],
       [ 1646,    21,     0]])

In [36]:
cm

array([[11312, 21323, 11295],
       [ 2421, 14985,  1108],
       [  303,   167,  1197]])

In [49]:
cm

array([[40064,  3771,    95],
       [15459,  2959,    96],
       [ 1580,    61,    26]])

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.70      0.91      0.79     43930
          1       0.44      0.16      0.23     18514
          2       0.12      0.02      0.03      1667

avg / total       0.61      0.67      0.61     64111



In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.81      0.26      0.39     43930
          1       0.41      0.81      0.55     18514
          2       0.09      0.72      0.16      1667

avg / total       0.67      0.43      0.43     64111



In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
svd_matrix_test =svd_matrix[320552:,:]

In [ ]:
svd_matrix_test.shape

In [ ]:
y_pred = classifier.predict(svd_matrix_test)

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] != 0 and y_pred[i] != 1:
        print(y_pred[i])

In [ ]:
y_pred

In [ ]:
y_pred_final = [0] * len(y_pred)
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        y_pred_final[i] ='unrelated'
    elif y_pred[i] == 1:
        y_pred_final[i] = 'agreed'
    elif y_pred[i] == 2:
        y_pred_final[i] = 'disagreed'

In [ ]:
for i in range(len(y_pred_final)):
    if y_pred_final[i] != 'unrelated':
        print(y_pred_final[i])

In [ ]:
y_pred_sub = [0] * len(y_pred)
for i in range(len(y_pred)):
    y_pred_sub[i] = [0] * 2

In [ ]:
for i in range(len(y_pred_sub)):
    y_pred_sub[i][0] = test_df.iloc[i,0]

In [ ]:
for i in range(len(y_pred_sub)):
    y_pred_sub[i][1] = y_pred_final[i]

In [ ]:
out = pd.DataFrame(y_pred_sub, columns = ["Id", "Category"])

In [ ]:
out

In [ ]:
out.to_csv('/Users/Administrator/Downloads/FakeNewsWSDM/output4.csv', index = False)

In [ ]:
out.to_csv?